In [359]:
import cv2
import re
import os
import numpy as np
from statistics import median

In [360]:
nImages = None
imageSetNumber = 1
kFirstMatches = 2

nKeyPointsInitial = 15000
goodMatchRatio = 0.7

histogramEqualizationWIndowSize = (2,2)
xParameter=2.0

BlurTuple = (5,5)

resizeFactor = 5
resizeTuple = (4160//resizeFactor, 2340//resizeFactor)


k = 100
ScaleCorrection = [[k,0,0],[0,k,0],[0,0,1]]



In [361]:
imageSetPath = "./In-Sample/"+str(imageSetNumber)+"/"


images = [ cv2.imread((imageSetPath+image)) for image in os.listdir(imageSetPath) if image[-3:]=="jpg"]
nImages = len(images)
# for i in range(len(images)) :
#     images[i] = cv2.cvtColor(images[i], cv2.COLOR_RGB2GRAY)

# print(len(images))



for i in range(len(images)):
    blur = cv2.blur(images[i], BlurTuple)
    images[i] = cv2.resize(blur,resizeTuple)
    


In [362]:
def Laplacian_Pyramid_Blending_with_mask(A, B, m, num_levels = 6):
    # assume mask is float32 [0,1]

    
    k = 2**num_levels
    A = A[:(A.shape[0]//k)*k, :(A.shape[1]//k)*k,:]
    B = B[:(B.shape[0]//k)*k, :(B.shape[1]//k)*k,:]
    m = m[:(m.shape[0]//k)*k, :(m.shape[1]//k)*k,:]

    
    # generate Gaussian pyramid for A,B and mask
    GA = A.copy()
    GB = B.copy()
    GM = m.copy()
    gpA = [GA]
    gpB = [GB]
    gpM = [GM]
    for i in range(num_levels):
        GA = cv2.pyrDown(GA)
        GB = cv2.pyrDown(GB)
        GM = cv2.pyrDown(GM)
        gpA.append(np.float32(GA))
        gpB.append(np.float32(GB))
        gpM.append(np.float32(GM))

    # generate Laplacian Pyramids for A,B and masks
    lpA  = [gpA[num_levels-1]] # the bottom of the Lap-pyr holds the last (smallest) Gauss level
    lpB  = [gpB[num_levels-1]]
    gpMr = [gpM[num_levels-1]]
    for i in range(num_levels-1,0,-1):
        # Laplacian: subtarct upscaled version of lower level from current level
        # to get the high frequencies
        LA = np.subtract(gpA[i-1], cv2.pyrUp(gpA[i]))
        LB = np.subtract(gpB[i-1], cv2.pyrUp(gpB[i]))
        lpA.append(LA)
        lpB.append(LB)
        gpMr.append(gpM[i-1]) # also reverse the masks

    # Now blend images according to mask in each level
    LS = []
    for la,lb,gm in zip(lpA,lpB,gpMr):
        ls = (la * gm + lb * (255 - gm))/255
        LS.append(ls)

    # now reconstruct
    ls_ = LS[0]
    
    for i in range(1,num_levels):
        ls_ = cv2.pyrUp(ls_)
        print(ls_.dtype, LS[i].dtype)
        ls_ = (cv2.add(ls_, np.float32(LS[i])))

    return ls_

# if __name__ == '__main__':
#     A = cv2.imread("input1.png",0)
#     B = cv2.imread("input2.png",0)
#     m = np.zeros_like(A, dtype='float32')
#     m[:,A.shape[1]/2:] = 1 # make the mask half-and-half
#     lpb = Laplacian_Pyramid_Blending_with_mask(A, B, m, 5)
#     cv2.imwrite("lpb.png",lpb)

In [363]:
def getMatches(img1, img2):
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck = False)
    orb = cv2.ORB_create(nfeatures=nKeyPointsInitial)

    keypoints1, descriptors1 = orb.detectAndCompute(img1, None)
    keypoints2, descriptors2 = orb.detectAndCompute(img2, None)
    
    matches = bf.knnMatch(descriptors1, descriptors2, k=kFirstMatches)
    matches = selectGoodMatches(matches)
    
#     print("matches list is:", matches)
    
    return matches, keypoints1, keypoints2

In [364]:
def getOrdering(images):
    
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck = False)
    orb = cv2.ORB_create(nfeatures=nKeyPointsInitial)
    splitImages = []
    for image in images:
        cols = len(image[0])
        splitImages.append(image[:,0:(cols//2),:])
        splitImages.append(image[:,(cols//2):cols,:])
    
    images = splitImages
    nImages=len(images)
    
    
    maxIndices = None, None
    maxMatches = []
    

    matchDictionary = {}
    for i in range(nImages):
        matchDictionary[i] = {}
    
    
    keypoints = []
    descriptors = []
    
    for image in images :
        keypoints_orb, descriptors_orb = orb.detectAndCompute(image, None)
        keypoints.append(keypoints_orb)
        descriptors.append(descriptors_orb)
    
    
    
    for i in range(nImages):
        print("\n")
        for j in range(i+1,nImages):
            matches = bf.knnMatch(descriptors[i], descriptors[j], k=kFirstMatches)
            matches = selectGoodMatches(matches)
            matchDictionary[i][j] = matches
            matchDictionary[j][i] = matches
            print(i,j,len(matchDictionary[i][j]))
        
            
    
    
    imageClusters = [[2*i, 2*i+1] for i in range(nImages//2)]
    
    for n in range(len(imageClusters),1,-1):
        
        closestClusters = None, None
        maxMatch = 0
        
        print(imageClusters)
        for i in range(len(imageClusters)):
            img1 = imageClusters[i][-1]
            for j in range(len(imageClusters)):
                if(i!=j):
                    img2 = imageClusters[j][0]
                    if(len(matchDictionary[img1][img2])>maxMatch):
                        maxMatch = len(matchDictionary[img1][img2])
                        closestClusters = i, j
#         print(closestClusters)
        ind1, ind2 = closestClusters
        imageClusters[ind1]+=imageClusters[ind2]
        del imageClusters[ind2]
    
    returnIndices = [imageClusters[0][i]//2 for i in range(0,len(imageClusters[0]),2)]
    print(returnIndices)
    return returnIndices, matchDictionary

In [365]:
def selectGoodMatches(matches):
    # Apply ratio test
    good = []
    for m in matches:
        if m[0].distance < goodMatchRatio*m[1].distance:
            good.append(m[0])
    matches = np.asarray(good)
    return matches

In [383]:
def rgbEqualize(img):
    img_yuv = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)

    # equalize the histogram of the Y channel
#     img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])

    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=histogramEqualizationWIndowSize)
    img_yuv[:,:,0] = clahe.apply(img_yuv[:,:,0])
    # convert the YUV image back to RGB format
    img_output = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)
    return img_output

In [384]:
def warpTwoImages(img1, img2):
    '''warp img2 to img1 with homograph H'''
    
    matches, keypoints1, keypoints2 = getMatches(img1, img2)
    
    print("shape of matches is:", matches.shape)
    
    src_pts = np.float32([ keypoints1[m.queryIdx].pt for m in matches ]).reshape(-1,1,2)
    dst_pts = np.float32([ keypoints2[m.trainIdx].pt for m in matches ]).reshape(-1,1,2)

    print(len(src_pts),len(dst_pts))
    H, mask1 = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC,xParameter)
#     H = np.linalg.inv(ScaleCorrection)*H*ScaleCorrection
    
    
    
#     pts1 = np.float32([[0,0],[0,h1],[w1,h1],[w1,0]])
#     pts2 = np.float32([[0,0],[0,h2],[w2,h2],[w2,0]])
#     pts2_ = cv2.getAffineTransform(pts2, H)



    h1,w1 = img1.shape[:2]
    h2,w2 = img2.shape[:2]
    pts1 = np.float32([[0,0],[0,h1],[w1,h1],[w1,0]]).reshape(-1,1,2)
    pts2 = np.float32([[0,0],[0,h2],[w2,h2],[w2,0]]).reshape(-1,1,2)
    pts2_ = cv2.perspectiveTransform(pts2, H)
    transformedPoints = pts2_.reshape(4,2)
#     print(pts2_.reshape(4,2))
    pts = np.concatenate((pts1, pts2_), axis=0)
    [xmin, ymin] = np.int32(pts.min(axis=0).ravel() - 0.5)
    [xmax, ymax] = np.int32(pts.max(axis=0).ravel() + 0.5)
    t = [-xmin,-ymin]
    Ht = np.array([[1,0,t[0]],[0,1,t[1]],[0,0,1]]) # translate
    
    
    translatedPoints = [[point[0]+t[0],point[1]+t[1]] for point in transformedPoints]
#     for point in transformedPoints:
#         x,y = point
#         temparray = [x+t[0], y+t[1]]
#         translatedPoints.append(temparray)

    
    warpedImage = cv2.warpPerspective(img2, Ht.dot(H), (xmax-xmin, ymax-ymin))
    
    
    mask = np.zeros(warpedImage.shape, dtype=np.uint8)
    cv2.fillPoly(mask, np.array([translatedPoints], dtype=np.int32), (255, 255, 255))
    
    result = np.zeros(warpedImage.shape, dtype=np.uint8)
    result[t[1]:h1+t[1],t[0]:w1+t[0]] = img1
#     result[np.where(mask == 255)] = warpedImage[np.where(mask == 255)]
    
    A = warpedImage
    B = result
    m = mask
    lpb = Laplacian_Pyramid_Blending_with_mask(A, B, m, 7)
    result = lpb
    
    result = rgbEqualize(result)
    return result

In [385]:
def writeImages(images):
    for i in range(nImages):
        cv2.imwrite("log"+str(i)+".jpg",images[i])
        
        
writeImages(images)

In [386]:
def horizontalStitch(images):
    stitchOrder, matches = getOrdering(images)
    orderedImages = []
    for i in stitchOrder:
        orderedImages.append(images[i])
    centerImageIndex = len(orderedImages)//2
    print(centerImageIndex)
    
#     img = orderedImages[centerImageIndex]
#     for i in range(centerImageIndex+1,centerImageIndex+2):#len(orderedImages)):
#         img = warpTwoImages(img, orderedImages[i])
        
#     for i in range(centerImageIndex-1,centerImageIndex-2,-1):#len(orderedImages)):
#         img = warpTwoImages(img, orderedImages[i])
    
    
    rightStitchedImage = stitchFromRight(orderedImages[centerImageIndex], orderedImages[centerImageIndex+1:])
    leftStitchedImage = stitchFromLeft(rightStitchedImage, orderedImages[:centerImageIndex]) 
    
    return leftStitchedImage
#     return img

    
def stitchFromRight(image, images):
    if (len(images) == 1):
        return image
    warpedImg = warpTwoImages(image, images[0])
    return stitchFromRight(warpedImg, images[1:])
    
    
    
    
def stitchFromLeft(image, images):
    if (len(images) == 1):
        return image
    warpedImg = warpTwoImages(image, images[-1])
    return stitchFromLeft(warpedImg, images[:-1])
    
    
result = horizontalStitch(images)



0 1 12
0 2 2
0 3 9
0 4 8
0 5 11
0 6 345
0 7 1277


1 2 1656
1 3 10
1 4 522
1 5 9
1 6 11
1 7 333


2 3 4
2 4 426
2 5 9
2 6 8
2 7 370


3 4 1887
3 5 231
3 6 5
3 7 7


4 5 9
4 6 3
4 7 10


5 6 3
5 7 7


6 7 1


[[0, 1], [2, 3], [4, 5], [6, 7]]
[[0, 1], [2, 3, 4, 5], [6, 7]]
[[0, 1, 2, 3, 4, 5], [6, 7]]
[3, 0, 1, 2]
2
shape of matches is: (1367,)
1367 1367
float32 float32
float32 float32
float32 float32
float32 float32
float32 float32
float32 float32


error: OpenCV(4.1.1) /tmp/opencv-20190908-76835-1lnncvq/opencv-4.1.1/modules/imgproc/src/clahe.cpp:351: error: (-215:Assertion failed) _src.type() == CV_8UC1 || _src.type() == CV_16UC1 in function 'apply'


In [ ]:
cv2.imwrite("log.jpg",result)